In [5]:
from ast import List

from torch import tensor
from mytorch import Tensor
import numpy as np

from mytorch.tensor import Tensorable

from mytorch.layer import Layer
from mytorch.util import initializer

In [2]:

class Conv2d(Layer):
    def __init__(self, in_channels, out_channels, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), need_bias: bool = False, mode="xavier") -> None:
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.need_bias = need_bias
        self.weight: Tensor = None
        self.bias: Tensor = None
        self.initialize_mode = mode

        self.initialize()

    def forward(self, x: Tensor) -> Tensor:
        "TODO: implement forward pass"
        batch_size, _, height, width = x.shape

        out_height = (height + 2 * self.padding[0] - self.kernel_size[0]) // self.stride[0] + 1
        out_width = (width + 2 * self.padding[1] - self.kernel_size[1]) // self.stride[1] + 1

        # if self.padding is not None and self.padding!=(0,0):
        #     for batch_index in range(batch_size):
        #         for channel_index in range(channel_size):
        #             pass


        # # padding
        # if self.padding is not None and self.padding!=(0,0):
        #     batch_data= []
        #     for data in x.data:
        #         channel_data = []
        #         for channel in data:
        #             channel_data.append(zero_padding(channel))
        #         batch_data.append(channel_data)
        #     data= batch_data
        # else:
        #     data= x.data

        # x.__getitem__(slice(batch_index)).__getitem__(slice(channel_index))

        result = Tensor(np.zeros((batch_size, self.out_channels, out_height, out_width)))

        # conv
        for b in range(batch_size):
            for o in range(self.out_channels):
                for i in range(out_height):
                    for j in range(out_width):
                        x_start = i * self.stride[0]
                        x_end = x_start + self.kernel_size[0]
                        y_start = j * self.stride[1]
                        y_end = y_start + self.kernel_size[1]
                        output= (x[b, :, x_start:x_end, y_start:y_end].__mul__(self.weight[o, :, :, :])).sum()
                        # result[b, o, i, j] = output
                        result[b, o, i, j] = output

        if self.need_bias:
            result = result + b

        return result
    
    def initialize(self):
        "TODO: initialize weight by initializer function (mode)"
        self.weight = Tensor(
            data=initializer((self.out_channels, self.in_channels ,self.kernel_size[0], self.kernel_size[1]), mode=self.initialize_mode),
            requires_grad=True,            
        )

        "TODO: initialize bias by initializer function (zero mode)"
        if self.need_bias:
            self.bias = Tensor(
                data=initializer((1, self.outputs), mode='zero'),
                requires_grad=True,
            )

    def zero_grad(self):
        "TODO: implement zero grad"
        self.weight.zero_grad()
        if self.need_bias:
            self.bias.zero_grad()

    def parameters(self):
        "TODO: return weights and bias"
        if self.need_bias:
            return [self.weight, self.bias]
        return [self.weight]
    
    def __str__(self) -> str:
        return "conv 2d - total params: {} - kernel: {}, stride: {}, padding: {}".format(
                                                                                    self.kernel_size[0] * self.kernel_size[1],
                                                                                    self.kernel_size,
                                                                                    self.stride, self.padding)

def zero_padding(x, padding_size=(1,1)):
    data= np.pad(x, padding_size, mode='constant', constant_values=0)
    return data


a= Tensor([[[[1,3,1],[3,1,2]], [[1,3,1],[3,1,2]]], [[[2,2,2],[2,2,2]], [[1,3,1],[3,1,2]]]])
conv2d1=Conv2d(2, 3, (2,2))
print(conv2d1(a))


Tensor([[[[-1.38594823 -2.896912  ]]

  [[-2.35066937 -1.25284158]]

  [[ 3.62985098 -1.06146296]]]


 [[[-0.14924004 -3.77867331]]

  [[-2.3767605  -1.10116047]]

  [[ 2.56045073  0.58914021]]]], requires_grad=True)


In [6]:
class AvgPool2d():
    def __init__(self, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0)) -> None:
        super()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def forward(self, x: Tensor) -> Tensor:

        def zero_padding(x, padding_size=(1,1)):
            data= np.pad(x, padding_size, mode='constant', constant_values=0)
            return data
        
        "TODO: implement forward pass"
        batch_size, channel_numbers, H, W = x.shape
        H_out = (H + 2 * self.padding[0] - self.kernel_size[0]) // self.stride[0] + 1
        W_out = (W + 2 * self.padding[1] - self.kernel_size[1]) // self.stride[1] + 1

        if self.padding is not None and self.padding!=(0,0):
            batch_data= []
            for data in x.data:
                channel_data = []
                for channel in data:
                    channel_data.append(zero_padding(channel))
                batch_data.append(channel_data)
            data= batch_data
        else:
            data= x.data

        output_data = np.zeros((batch_size, channel_numbers, H_out, W_out))
        for n in range(batch_size):
            for c in range(channel_numbers):
                for h in range(H_out):
                    for w in range(W_out):
                        output_data[n, c, h, w] = np.mean(data[n, c, h * self.stride[0] + self.padding[0] : h * self.stride[0] + self.padding[0] + self.kernel_size[0],
                                                        w * self.stride[1] + self.padding[1] : w * self.stride[1] + self.padding[1] + self.kernel_size[1]])
        
        return Tensor(output_data, x.requires_grad, x.depends_on)
    
    def __str__(self) -> str:
        return "avg pool 2d - kernel: {}, stride: {}, padding: {}".format(self.kernel_size, self.stride, self.padding)
    
a= Tensor([[[[1,3,1],[3,1,2]], [[1,3,1],[3,1,2]]], [[[2,2,2],[2,2,2]], [[1,3,1],[3,1,2]]]])
print(a.data.ndim)
# print(a)
# my_average_pool= AvgPool2d(kernel_size=(2,2))
# print(my_average_pool.forward(a))

4


In [2]:
from mytorch import Tensor
from mytorch.layer import Conv2d

a=Tensor([[[[1,3,1],[3,1,2]], [[1,3,1],[3,1,2]]], [[[2,2,2],[2,2,2]], [[1,3,1],[3,1,2]]]])
# conv_layer = Conv2d(in_channels=2, out_channels=1, kernel_size=(2,2))

# # Perform the convolution operation
# output = conv_layer(a)

# print(output)

Tensor([[[[-3.85258372 -1.84446653]]]


 [[[-1.18781617 -2.27989021]]]], requires_grad=True)


In [4]:
import numpy as np
from mytorch import Tensor, Dependency

def flatten(x: Tensor) -> Tensor:
    """
    TODO: implement flatten. 
    this methods transforms a n dimensional array into a flat array
    hint: use numpy flatten
    """
    batch_data= []
    for data in x._data:
        batch_data.append(data.flatten())
    data= batch_data

    req_grad = x.requires_grad
    
    if req_grad:
        def grad_fn(grad: np.ndarray):
            return grad.reshape()

        depends_on = [Dependency(x, grad_fn)]
    else:
        depends_on = []
        
    depends_on = x.depends_on
    return Tensor(data=data, requires_grad=req_grad, depends_on=depends_on)

a=Tensor([[[[1,3,1],[3,1,2]], [[1,3,1],[3,1,2]]], [[[2,2,2],[2,2,2]], [[1,3,1],[3,1,2]]]])
print(a.shape)
b=flatten(a)
print(b)
print(b.data.reshape(a.shape))

(2, 2, 2, 3)
Tensor([[1 3 1 3 1 2 1 3 1 3 1 2]
 [2 2 2 2 2 2 1 3 1 3 1 2]], requires_grad=False)
[[[[1 3 1]
   [3 1 2]]

  [[1 3 1]
   [3 1 2]]]


 [[[2 2 2]
   [2 2 2]]

  [[1 3 1]
   [3 1 2]]]]


In [22]:
import numpy as np
np.random.seed(42)
A = np.random.randint(0, 10, size=(1, 4, 2))
B = np.random.randint(0, 10, size=(4, 5, 3))
# C = B[0].reshape(1,2,3)
# print("A:\n{}, shape={}\nB:\n{}, shape={}\nC:\n{}, shape={}".format(
print("A:\n{}, shape={}\nB:\n{}, shape={}\n".format(
# A, A.shape, B, B.shape, C, C.shape))
A, A.shape, B, B.shape))
D = np.matmul(A, B)
# E = np.matmul(A, C)

print("Product D:\n{}, shape={}".format(D, D.shape))
# print("Product E:\n{}, shape={}".format(E, E.shape))


A:
[[[6 3]
  [7 4]
  [6 9]
  [2 6]]], shape=(1, 4, 2)
B:
[[[7 4 3]
  [7 7 2]
  [5 4 1]
  [7 5 1]
  [4 0 9]]

 [[5 8 0]
  [9 2 6]
  [3 8 2]
  [4 2 6]
  [4 8 6]]

 [[1 3 8]
  [1 9 8]
  [9 4 1]
  [3 6 7]
  [2 0 3]]

 [[1 7 3]
  [1 5 5]
  [9 3 5]
  [1 9 1]
  [9 3 7]]], shape=(4, 5, 3)



ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 2)